# フラクタル

樹木の枝の付き方、雪の結晶の形、海岸線や山の稜線の形など、自然界にはその一部分を拡大すると元の形と同じようなものが現れる自己相似（フラクタル; fractal）という性質を持つものが多い。今回は、いくつかのフラクタル図形を描画してみよう。

## コッホ曲線（再帰処理）

有限の長さの線分を三等分し、分割した 2 点で正三角形を描いて底辺を消す。この操作を何度も繰り返してできあがるのがコッホ曲線（Koch curve）である（図 1）。

![コッホ曲線](https://tueda.github.io/PS2020SS/notebooks/images/koch.png)
図 1. コッホ曲線

図 2 のように、ある線分が点 $P_1$ と $P_2$ によって与えられたとき、点 $A$、$B$、$C$ をどう求めるかは単純に平面幾何学の問題である。いま、

$$
\vec{d} = \overrightarrow{P_1 P_2} = 
\begin{pmatrix}
x_2 - x_1 \\
y_2 - y_1
\end{pmatrix} ,
\tag{1}
$$

とし、$\vec{d}$ を反時計回りに $90^\circ$ 回転させたものを $\vec{d}_\perp$ としよう：

$$
\vec{d}_\perp = 
\begin{pmatrix}
\displaystyle \cos\left(\frac{\pi}{2}\right) &
\displaystyle -\sin\left(\frac{\pi}{2}\right) \\
\displaystyle \sin\left(\frac{\pi}{2}\right) &
\displaystyle \phantom{-} \cos\left(\frac{\pi}{2}\right)
\end{pmatrix}
\vec{d} =
\begin{pmatrix}
y_1 - y_2 \\
x_2 - x_1
\end{pmatrix} .
\tag{2}
$$

すると、図 2 から明らかなように、

$$
\begin{align}
A &= P_1 + \frac{1}{3} \vec{d} ,
\tag{3a} \\
B &= P_1 + \frac{1}{2} \vec{d} + \frac{\sqrt{3}}{6} \vec{d}_\perp ,
\tag{3b} \\
C &= P_1 + \frac{2}{3} \vec{d} ,
\tag{3c}
\end{align}
$$

である。

![線分の分割](https://tueda.github.io/PS2020SS/notebooks/images/koch_geom.png)
図 2. 線分の分割

いま描きたいコッホ曲線は階層的な構造を持っているので、関数の再帰呼び出しと相性がよい。線分の分割を行う関数を作り、その関数からそれ自身を再帰的に呼び出すことによって、線分の分割を繰り返せばよい[<sup id="cite_ref-1">[1]</sup>](#cite_note-1)。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import sqrt

max_level = 6  # 再帰の深さ

# 折れ線グラフを描くための点を保存するリスト。
xpoints = []
ypoints = []


def koch(x1, y1, x2, y2, level):  # (x1,y1)から(x2,y2)の間のコッホ曲線
    if level == 0:
        # levelが0であれば点を追加する。
        if len(xpoints) == 0:
            # リストが空のとき（つまり最初のみ）
            xpoints.append(x1)
            ypoints.append(y1)
        xpoints.append(x2)
        ypoints.append(y2)
    else:
        # levelが0より大きければ、点A、B、Cの位置を求め、
        # 四つの線分に分割する(level - 1を渡す)。
        dx = x2 - x1
        dy = y2 - y1
        
        xa = x1 + 1 / 3 * dx
        ya = y1 + 1 / 3 * dy
        
        xb = x1 + 1 / 2 * dx - sqrt(3) / 6 * dy
        yb = y1 + 1 / 2 * dy + sqrt(3) / 6 * dx
        
        xc = x1 + 2 / 3 * dx
        yc = y1 + 2 / 3 * dy
        
        koch(x1, y1, xa, ya, level - 1)
        koch(xa, ya, xb, yb, level - 1)
        koch(xb, yb, xc, yc, level - 1)
        koch(xc, yc, x2, y2, level - 1)


# 実際にkoch関数を呼ぶ。
koch(0, 0, 1, 0, max_level)

# 折れ線グラフを描く。
fig, ax = plt.subplots()
ax.plot(xpoints, ypoints)
ax.set_aspect('equal')
plt.show()

## 練習問題

(1) 上のプログラムでは $(0,0) \to (1, 0)$ の間にコッホ曲線を描いた。これに加えて、 $(1, 0) \to (1 / 2, -\sqrt{3}/2)$ 、$(1 / 2, -\sqrt{3}/2) \to (0, 0)$ の間の二つのコッホ曲線を追加してみよう（つまり `koch` 関数 を 2 回追加で呼ぶ）。できた図形はコッホ雪片（Koch snowflake）と呼ばれる[<sup id="cite_ref-2">[2]</sup>](#cite_note-2)。

(2) コッホ曲線の一部を拡大しても同じ形が現れる、フラクタル構造を確認してみよう。これには、[Axes オブジェクト](https://matplotlib.org/api/axes_api.html#the-axes-class) の [set_xlim メソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_xlim.html)や [set_ylim メソッド](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_ylim.html)を使って、横軸と縦軸の範囲を指定してやればよい（例えば、 $0.10 \le x \le 0.23$ と $0.05 \le y \le 0.10$）。

## コッホ曲線（反復関数系）

今度はコッホ曲線を違った方法で描いてみよう。次のような二つの二次元写像を繰り返すことを考える（反復関数系；iterated function system; IFS）：

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} x_n + \frac{\sqrt{3}}{6} y_n , \\
y_{n+1} = \displaystyle \frac{\sqrt{3}}{6} x_n - \frac{1}{2} y_n ,
\end{cases}
\tag{4a} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} x_n - \frac{\sqrt{3}}{6} y_n + \frac{1}{2} , \\
y_{n+1} = \displaystyle - \frac{\sqrt{3}}{6} x_n - \frac{1}{2} y_n + \frac{\sqrt{3}}{6} .
\end{cases}
\tag{4b}
\end{align}
$$

ただし、各ステップで写像①と②をそれぞれ確率 $1/2$ で、どちらか一つを選んで適用するものとする。このようにしてできる点の集合を**散布図**によって描画する（カオスゲーム；chaos game）。初期位置は $(x_0,y_0) = (0, 0)$ としてみよう。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
from math import sqrt

n = 10000  # ステップ数

xpoints = []  # 各ステップでの位置を保存するリスト
ypoints = []

# 初期位置
x = 0
y = 0
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 二つの写像のどちらかをランダムに選んで適用する。
    r = random.random()
    if r < 0.5:
        x_new = 1 / 2 * x + sqrt(3) / 6 * y
        y_new = sqrt(3) / 6 * x - 1 / 2 * y
    else:
        x_new = 1 / 2 * x - sqrt(3) / 6 * y + 1 / 2
        y_new = - sqrt(3) / 6 * x - 1 / 2 * y + sqrt(3) / 6
        
    x = x_new
    y = y_new

    # 各ステップでの位置をリストに追加
    xpoints.append(x)
    ypoints.append(y)

# 散布図を描く。
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=0.5)
ax.set_aspect('equal')
plt.show()

## 練習問題

(3) 式 (4a) と (4b) の係数を一つでも変更すると違う図形となることを確認してみよう（演習課題 (3) 参照）。

## シェルピンスキーのギャスケット

次の三つの写像からなる反復関数系を考える。

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} x_n  , \\
y_{n+1} = \displaystyle \frac{1}{2} y_n ,
\end{cases}
\tag{5a} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} (x_n + 2) , \\
y_{n+1} = \displaystyle \frac{1}{2} y_n ,
\end{cases}
\tag{5b} \\
\text{③} \,
&\begin{cases}
x_{n+1} = \displaystyle \frac{1}{2} (x_n + 1) , \\
y_{n+1} = \displaystyle \frac{1}{2} (y_n + 1) .
\end{cases}
\tag{5c}
\end{align}
$$

ただし、①から③の写像をそれぞれ確率 $1/3$ で選ぶ。初期位置は $(x_0,y_0) = (1/2, 1/2)$ としてみよう。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random

n = 10000  # ステップ数

xpoints = []  # 各ステップでの位置を保存するリスト
ypoints = []

# 初期位置
x = 1 / 2
y = 1 / 2
xpoints.append(x)
ypoints.append(y)

for _ in range(n):
    # 三つの写像のどれかを適用する。
    r = random.random()
    if r < 1 / 3:
        x_new = 1 / 2 * x
        y_new = 1 / 2 * y
    elif r < 2 / 3:
        x_new = 1 / 2 * (x + 2)
        y_new = 1 / 2 * y
    else:
        x_new = 1 / 2 * (x + 1)
        y_new = 1 / 2 * (y + 1)
        
    x = x_new
    y = y_new

    # 各ステップでの位置をリストに追加
    xpoints.append(x)
    ypoints.append(y)

# 散布図を描く。
fig, ax = plt.subplots()
ax.scatter(xpoints, ypoints, s=0.5)
ax.set_aspect('equal')
plt.show()

この図形はシェルピンスキーのギャスケット（Sierpiński gasket）と呼ばれる[<sup id="cite_ref-3">[3]</sup>](#cite_note-3)。

## 演習課題

(1) 次の四つの写像からなる反復関数系を考える。

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = 0.8 x_n + 0.10 , \\
y_{n+1} = 0.8 y_n + 0.06 ,
\end{cases}
\tag{6a} \\
\text{②} \,
&\begin{cases}
x_{n+1} = 0.5 x_n + 0.250 , \\
y_{n+1} = 0.5 y_n + 0.465 ,
\end{cases}
\tag{6b} \\
\text{③} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.355 x_n - 0.355 y_n + 0.266 , \\
y_{n+1} = \phantom{-} 0.355 x_n + 0.355 y_n + 0.078 ,
\end{cases}
\tag{6c} \\
\text{④} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.355 x_n + 0.355 y_n + 0.378 , \\
y_{n+1} = - 0.355 x_n + 0.355 y_n + 0.434 .
\end{cases}
\tag{6d}
\end{align}
$$

ただし、①から④の写像を選ぶ確率はそれぞれ① 0.5、② 0.168、③ 0.166、④ 0.166、初期位置は $(x_0,y_0) = (0.5, 0.5)$ とする。これらの写像を繰り返して得られる点の集合よりフラクタル図形を描画せよ。

In [ ]:
# 課題解答11.1  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import random




(2) 次の四つの写像からなる反復関数系を考える。

$$
\begin{align}
\text{①} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.80 x_n + 0.05 y_n + 30 , \\
y_{n+1} = - 0.05 x_n +0.80 y_n + 80,
\end{cases}
\tag{7a} \\
\text{②} \,
&\begin{cases}
x_{n+1} = \phantom{-} 0.05 x_n - 0.30 y_n + 185 , \\
y_{n+1} = \phantom{-} 0.25 x_n + 0.15 y_n + 70,
\end{cases}
\tag{7b} \\
\text{③} \,
&\begin{cases}
x_{n+1} = - 0.05 x_n + 0.38 y_n + 150 , \\
y_{n+1} = \phantom{-} 0.26 x_n + 0.15 y_n + 50,
\end{cases}
\tag{7c} \\
\text{④} \,
&\begin{cases}
x_{n+1} = \phantom{-} 170 , \\
y_{n+1} = \phantom{-} 0.20 x_n + 0.20 y_n + 25 .
\end{cases}
\tag{7d}
\end{align}
$$

ただし、①から④の写像を選ぶ確率はそれぞれ① 0.73、② 0.13、③ 0.12、④ 0.02、初期位置は $(x_0,y_0) = (200, 200)$ とする。これらの写像を繰り返して得られる点の集合よりフラクタル図形を描画せよ。

In [ ]:
# 課題解答11.2  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import random




(3) 今回用いた写像の一般形は、平行移動を伴う線形写像

$$
\begin{cases}
x_{n+1} = a_{11} x_n + a_{12} y_n + b_1 , \\
y_{n+1} = a_{21} x_n + a_{22} y_n + b_2 ,
\end{cases}
\tag{8}
$$

であり、アフィン写像（affine map）と呼ばれる。コッホ曲線を反復関数系で描画した例題プログラムのように、いま**二種類**のアフィン写像を用い、その係数を適宜修正してオリジナルの**フラクタル図形**を描画してみよう。ただし、二つの写像を選ぶ確率はそれぞれ **1/2 ずつに固定**することにする。いくつか試して、気に入ったデザインのもののを提出せよ。

In [ ]:
# 課題解答11.3  <-- 提出する際に、この行を必ず含めること。

%matplotlib inline
import matplotlib.pyplot as plt
import random
# 使いたければmathモジュールの関数を適宜インポートせよ。




## 参考

式 (8) は、拡大係数行列を用いて表した

$$
\begin{pmatrix}
x_{n+1} \\
y_{n+1} \\
1
\end{pmatrix}
=
\begin{pmatrix}
a_{11} & a_{12} & b_1 \\
a_{21} & a_{22} & b_2 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
x_n \\
y_n \\
1
\end{pmatrix} ,
\tag{9}
$$

と等価である。このような行列は（正則であれば）群を成し、座標変換を表すのに都合がよい。任意のアフィン変換は回転、拡大縮小、平行移動を表す行列の積として表される[<sup id="cite_ref-4">[4]</sup>](#cite_note-4)[<sup id="cite_ref-5">[5]</sup>](#cite_note-5)。

$$
\begin{align}
\text{Rotation}(\theta) &=
\begin{pmatrix}
\cos(\theta) & - \sin(\theta) & 0 \\
\sin(\theta) & \phantom{-} \cos(\theta) & 0 \\
0 & 0 & 1
\end{pmatrix} ,
\tag{10a} \\
\text{Scaling}(s_x, s_y) &=
\begin{pmatrix}
s_x & 0 & 0 \\
0 & s_y & 0 \\
0 & 0 & 1
\end{pmatrix} ,
\tag{10b} \\
\text{Translation}(t_x, t_y) &=
\begin{pmatrix}
1 & 0 & t_x \\
0 & 1 & t_y \\
0 & 0 & 1
\end{pmatrix} .
\tag{10c}
\end{align}
$$

三次元空間への拡張も容易である。複数のアフィン変換の合成も行列の積によって一つにまとめることができるなど非常に便利であり、コンピューターグラフィックスの分野などで多用される。

行列の積によるアフィン写像の合成の例として、式 (4a) と (4b) の写像はそれぞれ、次のような行列の積で表される。

$$
\begin{pmatrix}
\displaystyle \cos\left(\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle -\sin\left(\frac{\pi}{6}\right) &
0 \\
\displaystyle \sin\left(\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle \phantom{-} \cos\left(\frac{\pi}{6}\right) & 
0 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \frac{1}{\sqrt{3}} & 0 & 0 \\
0 & \displaystyle - \frac{1}{\sqrt{3}} & 0 \\
0 & 0 & 1
\end{pmatrix}
=
\begin{pmatrix}
\displaystyle \frac{1}{2} &
\displaystyle \frac{\sqrt{3}}{6} & 0 \\
\displaystyle \frac{\sqrt{3}}{6} &
\displaystyle - \frac{1}{2} & 0 \\
0 & 0 & 1
\end{pmatrix} ,
\tag{11a}
$$

$$
\begin{pmatrix}
1 &
0 &
\displaystyle \frac{1}{2} \vphantom{\frac{1}{\sqrt{3}}} \\
0 &
1 & 
\displaystyle \frac{\sqrt{3}}{6} \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \cos\left(-\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle -\sin\left(-\frac{\pi}{6}\right) &
0 \\
\displaystyle \sin\left(-\frac{\pi}{6}\right) \vphantom{\frac{1}{\sqrt{3}}} &
\displaystyle \phantom{-} \cos\left(-\frac{\pi}{6}\right) & 
0 \\
0 & 0 & 1
\end{pmatrix}
\begin{pmatrix}
\displaystyle \frac{1}{\sqrt{3}} & 0 & 0 \\
0 & \displaystyle - \frac{1}{\sqrt{3}} & 0 \\
0 & 0 & 1
\end{pmatrix}
=
\begin{pmatrix}
\displaystyle \frac{1}{2} &
\displaystyle - \frac{\sqrt{3}}{6} &
\displaystyle \frac{1}{2} \\
\displaystyle - \frac{\sqrt{3}}{6} &
\displaystyle - \frac{1}{2} &
\displaystyle \frac{\sqrt{3}}{6} \\
0 & 0 & 1
\end{pmatrix} .
\tag{11b}
$$

これらは、図 2 の$\triangle P_1 P_2 B$ を、それぞれ $\triangle P_1 B A$ と $\triangle B P_2 C$ に移す写像となっている。

## 脚注

<span id="cite_note-1">1.</span> [^](#cite_ref-1)
分割操作を無限に繰り返して得られるのがコッホ曲線であるが、当然、計算機上では有限回の繰り返しで止めるという近似を行うことになる。

<span id="cite_note-2">2.</span> [^](#cite_ref-2)
コッホ曲線の分割操作を 1 回行うと、線分の全長は $4/3$ 倍になる。これを無限回繰り返すことになるので、コッホ曲線の長さは無限大である。よって、コッホ曲線を三つつないだ図形であるコッホ雪片の周の長さも無限大となる。一方、面積については、$n$ 回目の操作によって面積が $(1/3)^{2n} \sqrt{3} \, / 4$ である $3 \times 4^{n-1}$ 個の正三角形の分だけ増えるから、もとの三角形の面積とこれらを（無限個）すべて足し合わせて、$2\sqrt{3} \, / 5$ と求まる。

<span id="cite_note-3">3.</span> [^](#cite_ref-3)
次回またこの図形にお目にかかるかもしれない。

<span id="cite_note-4">4.</span> [^](#cite_ref-4)
剪断（せんだん）と呼ばれる写像もあるが、これは特異値分解（singular value decomposition; SVD）という行列分解によって、いくつかの拡大縮小と回転の積へと分解できる。

<span id="cite_note-5">5.</span> [^](#cite_ref-5)
これを見ていると、反復関数系に $\displaystyle \begin{vmatrix} a_{11} & a_{12} \\ a_{21} & a_{22} \end{vmatrix}$ の絶対値が $1$ より大きいような写像を入れると、それは拡大変換を含んでいるのだから、原点からの距離がどんどん大きくなってしまってまずそうだ（固定点がなく収束しない）、ということが分かるだろう。